In [1]:
import os
import pandas as pd
from glob import glob

# 1. 统计labelbox_sidewalk_legacy_data下的MP4文件数量
mp4_dir = 'labelbox_sidewalk_legacy_data'
mp4_files = [f for f in os.listdir(mp4_dir) if f.lower().endswith('.mp4')]
print(f"MP4文件数量: {len(mp4_files)}")

MP4文件数量: 1501


In [2]:
# 2. 读取continuous_videos.csv，找出不在labelbox_sidewalk_legacy_data中的视频
csv_path = 'data/continuous_videos.csv'
df = pd.read_csv(csv_path)

# 提取csv中的视频名（去掉扩展名，改为.mp4）
def mov_to_mp4(name):
    if name.lower().endswith('.mov'):
        return name[:-4] + '.mp4'
    return name
df['video_name_mp4'] = df['video_name'].apply(mov_to_mp4)

# 统计本地已有的mp4文件名集合
mp4_set = set(mp4_files)

# 找出不在本地的mp4
missing_mask = ~df['video_name_mp4'].isin(mp4_set)
missing_data_df = df[missing_mask].copy()
print(f"缺失的视频数量: {len(missing_data_df)}")
missing_data_df.head()

缺失的视频数量: 0


,video_name,collector_name,material,full_path,duration,video_name_mp4
